In [11]:
import os
import ssl

import sqlalchemy


def connect_tcp_socket() -> sqlalchemy.engine.base.Engine:
    """ Initializes a TCP connection pool for a Cloud SQL instance of Postgres. """
    # Note: Saving credentials in environment variables is convenient, but not
    # secure - consider a more secure solution such as
    # Cloud Secret Manager (https://cloud.google.com/secret-manager) to help
    # keep secrets safe.
    db_host = "127.0.0.1" # os.environ["INSTANCE_HOST"]  # e.g. '127.0.0.1' ('172.17.0.1' if deployed to GAE Flex)
    db_user = "postgres" # os.environ["DB_USER"]  # e.g. 'my-db-user'
    db_pass = "a0cp$ug1^o-[mjAs" # os.environ["DB_PASS"]  # e.g. 'my-db-password'
    db_name = "postgres" # os.environ["DB_NAME"]  # e.g. 'my-database'
    db_port = "5432" # os.environ["DB_PORT"]  # e.g. 5432

    pool = sqlalchemy.create_engine(
        # Equivalent URL:
        # postgresql+pg8000://<db_user>:<db_pass>@<db_host>:<db_port>/<db_name>
        sqlalchemy.engine.url.URL.create(
            drivername="postgresql+pg8000",
            username=db_user,
            password=db_pass,
            host=db_host,
            port=db_port,
            database=db_name,
        ),
        # ...
    )
    return pool

In [12]:
engine = connect_tcp_socket()
conn = engine.connect()

In [13]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship


class Base(DeclarativeBase):
    pass

class Query(Base):
    __tablename__ = "query"
    id: Mapped[int] = mapped_column(primary_key=True)
    text: Mapped[str] = mapped_column(String(30))
    responses: Mapped[List["Response"]] = relationship(
        back_populates="query", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"Query(id={self.id!r}, text={self.text!r})"

class Response(Base):
    __tablename__ = "response"
    id: Mapped[int] = mapped_column(primary_key=True)
    text: Mapped[str] = mapped_column(String(30))
    query_id: Mapped[int] = mapped_column(ForeignKey("query.id"))
    query: Mapped["Query"] = relationship(back_populates="responses")
    def __repr__(self) -> str:
        return f"Response(id={self.id!r}, text={self.text!r})"

In [14]:
Base.metadata.create_all(engine)

In [16]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    query = Query(
        text="Hello world!",
        responses=[Response(text="Answer...")],
    )
    queries = [query]
    session.add_all(queries)
    session.commit()

In [17]:
from sqlalchemy import select

session = Session(engine)

stmt = select(Query)

for query in session.scalars(stmt):
    print(query)

Query(id=1, text='Hello world!')


In [18]:
from sqlalchemy import select

session = Session(engine)

stmt = select(Response)

for response in session.scalars(stmt):
    print(response)

Response(id=1, text='Answer...')
